# Robot functions (to be completed)

In [6]:
# !pip install math3d

In [8]:
# import json
# import urx
# import robotiq_gripper

# ip1 = "192.168.2.207"   #ip of the UR robot to connect

# ip2 = ""   #???????

# gripper_1 = robotiq_gripper.RobotiqGripper()  # initialize the gripper
# gripper_2 = robotiq_gripper.RobotiqGripper()   #???????
# print("Connecting to grippers...")


# gripper_1.connect(ip1, 63352)                  # connect to the gripper
                                           
# robot_1 = urx.Robot(ip1, use_rt=True)            # connect to the UR robot


# gripper_2.connect(ip2, 63352)                  # connect to the gripper
                                           
# robot_2 = urx.Robot(ip2, use_rt=True)            # connect to the UR robot



# def move_top(robot_id, gripper_id , object_position):  
#     return ""

# def gripper_open(robot_id, gripper_id):
#     gripper_id.move_and_wait_for_pos(0,100,100)



############################# functions used in prompting 

def move_to_object(robot_id, gripper_id, object_position): # to move to the position of an object
    return " move_to_object(robot_id, gripper_id, object_position) " # for testing


def grasp_object(robot_id, gripper_id, object_position):  # to grasp an object
    return " grasp_object(robot_id, gripper_id, object_position) " # for testing

def connect(robot_id, gripper_id, object_1_position, object_2_position): # connect a part to another
    return " connect(robot_id, gripper_id, object_1_position, object_2_position)" # for testing

def hold_object(robot_id, gripper_id, object_position): # To apply a forcce on the base to avoid it moving
    return " hold_object(robot_id, gripper_id, object_position) " # for testing


In [12]:
# ! pip install langgraph
# ! pip install langchain_openai

## JSON file details extraction 

In [9]:
import sys
import json

class AssemblyEnvironment:
    def __init__(self, json_file):
        with open(json_file, 'r') as file:
            data = json.load(file)
        
        self.base = data['assembly']['base']
        self.base_num_pins=len(self.base["pins"])
        self.parts = data['assembly']['parts']
        self.num_parts = len(self.parts)
        self.base_position = data['assembly']['base']['position']

        # Dynamically create pin attributes
        for i in range(len(self.base["pins"])):
            pin_name = f"base_pin_{i + 1}"  # Create dynamic pin names
            setattr(self, pin_name, self.base["pins"][i])
        
        # Extract part details dynamically
        for i, part in enumerate(self.parts):
            part_id = part["id"]
            setattr(self, f"{part_id}_position", part["position"])
            setattr(self, f"{part_id}_target_position", part["target_position"])
            setattr(self, f"{part_id}_mounting_hole", part["mounting_hole"])
            setattr(self, f"{part_id}_grip_pin", part["grip_pin"])

        self.assembly_prompt  = lambda user_query: f""" 
You are an AI agent which task is to provide a detail plan for fully assembling an object of {self.num_parts + 1 } different components. 
The object has a base with { self.base_num_pins } pins where the  {self.num_parts} others parts which all contains amounting hole have to be mounted.
The object have to be assembled by two robot (Robot1_base_holder ,Robot2_assembler) manipulator each equipped with a gripper (gripper1, gripper2)and 
your goal is to provide a deatailed plan on how the to robots have to collaborate togother to solve the assembly task by strictly using 
the following functions to control the motions of the two robots: move_to_object(robot_id, gripper_id, object_position), 
grasp_object(robot_id, gripper_id, object_position),  hold_object(robot_id, gripper_id, object_position), 
connect(robot_id, gripper_id, object_1_position, object_2_position).

In order to answer use the following template:

START OF PLAN
 1 - [CHOICE:  Robot1_base_holder,Robot2_assembler]  should move to the position of the base using the function [CHOICE: move_to_object(robot_id, gripper_id, object_position), 
   grasp_object(robot_id, gripper_id, object_position),  hold_object(robot_id, gripper_id, object_position)]
 2 - [CHOICE:  Robot1_base_holder,Robot2_assembler]  should hold the base using the function [CHOICE: move_to_object(robot_id, gripper_id, object_position), 
grasp_object(robot_id, gripper_id, object_position),  hold_object(robot_id, gripper_id, object_position), 
connect(robot_id, gripper_id, object_1_position, object_2_position)]
 3 - [CHOICE:  Robot1_base_holder,Robot2_assembler]  shoul move to the positon of [CHOICE: part_1, part_2, ....] with the function  [CHOICE: move_to_object(robot_id, gripper_id, object_position), 
grasp_object(robot_id, gripper_id, object_position),  hold_object(robot_id, gripper_id, object_position), connect(robot_id, gripper_id, object_1_position, object_2_position)]
 4 - [CHOICE:  Robot1_base_holder,Robot2_assembler]  should move grasp [CHOICE: part_1, part_2, ....] with the function  [CHOICE: move_to_object(robot_id, gripper_id, object_position), 
grasp_object(robot_id, gripper_id, object_position),  hold_object(robot_id, gripper_id, object_position), 
connect(robot_id, gripper_id, object_1_position, object_2_position)]
 5 - [CHOICE:  Robot1_base_holder,Robot2_assembler]  should connect  [CHOICE: part_1, part_2, ....] to the pin  [CHOICE: pin_1, pin_2, ....] on the base with the function
[CHOICE: move_to_object(robot_id, gripper_id, object_position), grasp_object(robot_id, gripper_id, object_position),  hold_object(robot_id, gripper_id, object_position), 
connect(robot_id, gripper_id, object_1_position, object_2_position)]
 6 - .....
END OF PLAN


Rules:
1. your answer should explicitly start with 'START OF PLAN' and end it with 'END OF PLAN' .
2. When you see phrases like [CHOICE: choice1, choice2, ...], you should replace the entire phrase with only one of the choices listed.
4. Do not create or invent any other function; only use the provided predifined used functions  

Now here is the user given task : {user_query}"""    





if __name__ == "__main__":
    # Redirect stdout to a file
    original_stdout = sys.stdout  # Save a reference to the original standard output

    with open('output.txt', 'w') as f:
        sys.stdout = f  # Change the standard output to the file we created.

        # Create an instance of AssemblyEnvironment by providing the path to the JSON file
        assembly_env = AssemblyEnvironment('assembly_details.JSON')

        
        print("On The Base:")
        
        # Print the base and pins to verify
        for pin in assembly_env.base["pins"]:
            pin_id = pin["id"]
            pin_position = pin["position"]
            print(f"Pin {pin_id} Position on the base:", pin_position)  # Print pin position


        print("\n")
        print(f"The {assembly_env.num_parts} assembly parts:")
        
        # Accessing dynamic part attributes with detailed descriptions
        for part in assembly_env.parts:
            part_id = part["id"]
            print(f"Part {part_id}:")
            # print(f"{part_id} Position:", getattr(assembly_env, f"{part_id}_position"))  # Position of the part
            print(f"{part_id} Target Position:", getattr(assembly_env, f"{part_id}_target_position"))  # Target position of the part
            print(f"{part_id} Mounting Hole:", getattr(assembly_env, f"{part_id}_mounting_hole"))  # Mounting hole details of the part
            print(f"{part_id} Grip Pin:", getattr(assembly_env, f"{part_id}_grip_pin"))  # Grip pin position of the part
            print("-----------")
            
        sys.stdout = original_stdout  # Reset the standard output to its original value

print("All output has been saved to output.txt")
# print(assembly_env.assembly_prompt("hollllllllllllllllllllllllllla"))

All output has been saved to output.txt


## RAG System

In [10]:
# !pip install langchain-community
# !pip install sentence_transformers
# !pip install --upgrade chromadb


In [11]:
################################################################ RAG



from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

### Reading the txt files from source directory

loader = DirectoryLoader('./', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings

from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
# retriever = db.as_retriever(search_kwargs={"k": 4})
retriever = db.as_retriever(search_kwargs={"k": 1})




####################### testing RAG
query = "what is the Mounting Hole of part 2"
docs = retriever.invoke(query)
print(docs)

/home/koffi/anaconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


[Document(metadata={'source': 'output.txt'}, page_content="part_2 Mounting Hole: {'id': 2, 'position': {'x': 0, 'y': 0, 'alpha': 0}}")]


## Building agents

In [23]:
import os
from dotenv import load_dotenv

load_dotenv()


from langchain_openai import ChatOpenAI

# Now you can access your environment variables using os.environ
os.environ['OPENAI_API_KEY'] = ''
# os.environ['TAVILY_API_KEY'] = os.environ.get("TAVILY_API_KEY")


llm = ChatOpenAI(model="gpt-3.5-turbo")

In [24]:

# def generate_plan(state):
#     messages = state['messages']
#     question = messages[0] ## Fetching the user question

#     template = """Answer the question based only on the following context:
#     {context}

#     Question: {question}
#     """
#     prompt = ChatPromptTemplate.from_template(template)

#     retrieval_chain = (
#         {"context": retriever, "question": RunnablePassthrough()}
#         | prompt
#         | llm
#         | StrOutputParser()
#         )
#     result = retrieval_chain.invoke(question)
#     return result



def generate_plan_step1(user_query):
    print('-> ...Plan Generation... ->')
    complete_query = assembly_env.assembly_prompt( user_query) # to do
    response = llm.invoke(complete_query)
    plan = response.content
    return  plan


# def generate_plan_step2(plan):
#     print('-> ...Calling RAG... ->')
#     question = "Use the position x,y and alpha provided in the context to fill the arguments of the functions in the following plan: \n " + plan
#     print('Question:',question)

#     template = """Answer the question based only and strictly on the following context:
#     {context}

#     Question: {question}
#     """
#     prompt = ChatPromptTemplate.from_template(template)

#     retrieval_chain = (
#         {"context": retriever, "question": RunnablePassthrough()}
#         | prompt
#         | llm
#         | StrOutputParser()
#         )
#     result = retrieval_chain.invoke(question)
#     return result


def generate_plan_step2(plan, filename = 'output.txt' ):
    print('-> ...Calling RAG+-... ->')

    with open(filename, 'r') as file:
        context = file.read() #to check later 
    
    question = "Now here is the plan to fill up: \n " + plan
    print('Question:',question)

    template = """You will recieve a plan generated by an AI agent which contain a list of step that two robot manipulators 
have to follow in order to perform a user given task. The plan contain some functions and your goal is to replace the arguments 
of the function by their real values. All the reals variables values can be find in the following retrieved context:\n
    {context}

    Question: {question}
    """
    print('template:',template)
    prompt = ChatPromptTemplate.from_template(template)

    retrieval_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
        )
    result = retrieval_chain.invoke(question)
    return result



################################################################ Supervisor



from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

members = ["Robot1_base_holder", "Robot2_assembler"]
system_prompt = (
'''
You are a supervisor tasked with managing a collaboration between the following two robot workers: {members}. 
Given the following user request, respond with the worker to act next. Use Robot1_base_holder 
is assigned to it and Robot2_assembler when the task i assigned to it . Each worker will perform his task and 
respond with their results and status. 
When finished, respond with FINISH
'''
)

options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)


In [25]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI


def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [26]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [27]:
import json
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool

# @tool


# model_with_tools = model.bind(tools=[convert_to_openai_tool(multiply)])



# @tool
# def move_to_object_tool(robot_id, gripper_id, object_position):
#     return move_to_object(robot_id, gripper_id, object_position)

# @tool
# def grasp_object_tool(robot_id, gripper_id, object_position):
#     return grasp_object(robot_id, gripper_id, object_position)



# @tool
# def hold_object_tool(robot_id, gripper_id, object_position):
#     return hold_object(robot_id, gripper_id, object_position)


# @tool
# def connect_tool(robot_id, gripper_id, object_1_position, object_2_position):
#     return connect(robot_id, gripper_id, object_1_position, object_2_position)


from langchain_core.tools import BaseTool

class MoveToObjectTool(BaseTool):
    def _run(self, robot_id, gripper_id, object_position):
        # Implement the logic for moving the robot
        return move_to_object(robot_id, gripper_id, object_position)

class GraspObjectTool(BaseTool):
    def _run(self, robot_id, gripper_id, object_position):
        # Implement the logic for moving the robot
        return grasp_object(robot_id, gripper_id, object_position)


class HoldObjectTool(BaseTool):
    def _run(self, robot_id, gripper_id, object_position):
        # Implement the logic for moving the robot
        return hold_object(robot_id, gripper_id, object_position)


class ConnectTool(BaseTool):
    def _run(self, robot_id, gripper_id, object_1_position, object_2_position):
        # Implement the logic for moving the robot
        return connect(robot_id, gripper_id, object_1_position, object_2_position)









# @tool
# def move_to_object_tool(robot_id, gripper_id, object_position):
#     """Move the robot to the object position."""
#     return move_to_object(robot_id, gripper_id, object_position)

# @tool
# def grasp_object_tool(robot_id, gripper_id, object_position):
#     """Grasp the object at the specified position."""
#     return grasp_object(robot_id, gripper_id, object_position)

# @tool
# def hold_object_tool(robot_id, gripper_id, object_position):
#     """Hold the object at the specified position."""
#     return hold_object(robot_id, gripper_id, object_position)

# @tool
# def connect_tool(robot_id, gripper_id, object_1_position, object_2_position):
#     """Connect two objects at the specified positions."""
#     return connect(robot_id, gripper_id, object_1_position, object_2_position)




In [28]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI


def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [30]:
import operator
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import functools

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END


# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str


# research_agent = create_agent(llm, [tavily_tool], "You are a web researcher.")
# research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

# # NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION. PROCEED WITH CAUTION
# code_agent = create_agent(
#     llm,
#     [python_repl_tool],
#     "You may generate safe python code to analyze data and generate charts using matplotlib.",
# )
# code_node = functools.partial(agent_node, agent=code_agent, name="Coder")


# RAG_agent = create_agent(
#     llm,
#     [RAG],
#     "Use this tools when questions are related to Japan or of Sports category.",
# )
# rag_node = functools.partial(agent_node, agent=RAG_agent, name="RAG")

# workflow = StateGraph(AgentState)
# workflow.add_node("Researcher", research_node)
# workflow.add_node("Coder", code_node)
# workflow.add_node("RAG", rag_node)
# workflow.add_node("supervisor", supervisor_chain)

#######################################################################

# research_agent = create_react_agent(llm, tools=[tavily_tool])
# research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

# # NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION. PROCEED WITH CAUTION
# code_agent = create_react_agent(llm, tools=[python_repl_tool])
# code_node = functools.partial(agent_node, agent=code_agent, name="Coder")





# Remaining tooooooooooooooooooooooooooooooooooooooooooooooooollllllllllllllllllls
# @tool
# def multiply(first_number: int, second_number: int):
#     """Multiplies two numbers together."""
#     return first_number * second_number

# model_with_tools = model.bind(tools=[convert_to_openai_tool(move_to_object_tool),convert_to_openai_tool(grasp_object_tool),convert_to_openai_tool(hold_object_tool),convert_to_openai_tool(connect_tool)])





# robot1_agent= create_agent(llm, 
#                                  [convert_to_openai_tool(move_to_object_tool),convert_to_openai_tool(grasp_object_tool),convert_to_openai_tool(hold_object_tool),convert_to_openai_tool(connect_tool)],
#                                 'Use this tools when task is supposed to be done by Robot1_base_holder')

robot1_agent = create_agent(
    llm,
    [
        convert_to_openai_tool(MoveToObjectTool()),  # Ensure proper instantiation
        convert_to_openai_tool(GraspObjectTool()),
        convert_to_openai_tool(HoldObjectTool()),
        convert_to_openai_tool(ConnectTool()),
    ],
    'Use these tools when the task is supposed to be done by Robot1_base_holder'
)

robot1_node = functools.partial(agent_node, agent=robot1_agent, name="Robot1_base_holder")


# robot2_agent = create_agent(llm,
#                                   [convert_to_openai_tool(move_to_object_tool),convert_to_openai_tool(grasp_object_tool),convert_to_openai_tool(hold_object_tool),convert_to_openai_tool(connect_tool)],
#                                  'Use this tools when task is supposed to be done by Robot2_assembler')



robot2_agent = create_agent(
    llm,
    [
        convert_to_openai_tool(MoveToObjectTool()),  # Ensure proper instantiation
        convert_to_openai_tool(GraspObjectTool()),
        convert_to_openai_tool(HoldObjectTool()),
        convert_to_openai_tool(ConnectTool()),
    ],
    'Use this tools when task is supposed to be done by Robot2_assembler'
)
robot2_node = functools.partial(agent_node, agent=robot2_agent, name="Robot2_assembler")


ValueError: Value not declarable with JSON Schema, field: name='_callbacks_List[langchain_core.callbacks.base.BaseCallbackHandler]' type=BaseCallbackHandler required=True

In [ ]:

# workflow = Graph()

# workflow.add_node("Agent", function_1)
# workflow.add_node("tool", function_2)

# workflow.add_edge('Agent', 'tool')

# workflow.set_entry_point("Agent")
# workflow.set_finish_point("tool")

# app = workflow.compile()

workflow = StateGraph(AgentState)


workflow.add_node("generate_plan1", generate_plan_step1)
workflow.add_node("generate_plan2", generate_plan_step2)
workflow.add_edge('generate_plan1', 'generate_plan2')
workflow.add_node("supervisor", supervisor_chain)
workflow.add_node("Robot1_base_holder", robot1_node)
workflow.add_node("Robot2_assembler", robot2_node)




for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.set_entry_point("generate_plan1")

graph = workflow.compile()

In [ ]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="assemble a gripper of one base and two parts for me")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")